In [ ]:
!pip install -U git+https://github.com/bhbbbbb/f-final

  Cloning https://github.com/bhbbbbb/f-final to /tmp/pip-req-build-9eo9x3ee
  Running command git clone --filter=blob:none --quiet https://github.com/bhbbbbb/f-final /tmp/pip-req-build-9eo9x3ee
  Resolved https://github.com/bhbbbbb/f-final to commit 321743ce81cd122f20dd75f503e39849ee9863c9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


Data will be downloaded automatically and cached in the directory `dataset`.

Note that `merged_df` has been split with ratio train:val:test=0.7:0.2:0.1 (following hw3).

In [1]:
import numpy as np
import final
from final import data
# data.set_password(<password>)

## Baseline

In [2]:
def baseline_score(loads: list[dict]):
    """Score products using their indices in load sequences.

    E.g. Given loads(list of pids): [3, 10, 5, 3], then score(p3)==3, score(p10)==2, score(p5)==1.
        The rest products would have random negative scores.

    Args:
        loads (list[dict]): list of product ids
    """
    loads = [final.PID_MAPPING[load['item_id']] for load in loads]
    scores = np.random.rand(len(final.AVAILABLE_PRODUCT_IDS)) - 11.
    for i, pid in enumerate(loads):
        scores[pid] = i
    return scores

In [3]:
import pandas as pd
from pprint import pprint
row = []
for split, df in data.merged_df().items():
    scores = df['loads'].map(baseline_score)
    metrics = final.ranking_metrics(scores, df['y_true'], ks=[1, 5, 10, 20])
    metrics['split'] = split
    row.append(metrics)
pd.DataFrame(row).set_index('split')

,hr@1,hr@5,hr@10,hr@20,mrr@1,mrr@5,mrr@10,mrr@20,ndcg@1,ndcg@5,ndcg@10,ndcg@20
split,,,,,,,,,,,,
train,0.822683,0.865172,0.892568,0.899301,0.822683,1.065950,1.088366,1.091071,0.822683,0.835300,0.848772,0.851635
val,0.838833,0.864453,0.896562,0.905455,0.838833,1.094096,1.122622,1.125993,0.838833,0.839195,0.855356,0.859092
test,0.845378,0.874111,0.902801,0.908660,0.845378,1.095868,1.119642,1.122025,0.845378,0.849271,0.863468,0.866018


## TagEvaluator

In [12]:
product_tags = data.product_tags_v4()
product_tags # pid: list[tag(str)]

{5: ['愛文芒果',
  '芒果',
  '芒果冰',
  '產地嚴選愛文芒果(小果)',
  '水果',
  '水果禮盒',
  '送禮',
  '夏季水果',
  '濃郁香氣',
  '高甜度'],
 23: ['青花菜',
  '輕食',
  '十字花科',
  '抗癌蔬菜',
  '綠花椰',
  '飲食控制',
  '健身食譜',
  '低卡飲食',
  '沙拉',
  '低醣飲食'],
 24: ['花椰米', '健康飲食', '飲食控制', '健康料理', '備餐', '健身', '飲控', '熱量控制', '減肥', '鮮凍白花椰菜'],
 25: ['健康飲食', '飲食控制', '健康料理', '備餐', '健身', '飲控', '冷凍', '綜合蔬菜', '青花菜', '白花菜'],
 28: ['豬肉貢丸',
  '台灣豬後腿肉',
  '火鍋料理',
  '雲林新鮮豬肉',
  '貢丸湯料',
  '豬肉丸子',
  '原塊豬後腿肉',
  '純粹肉汁',
  '口感彈牙',
  '滋味豐富'],
 46: ['魚肉料理',
  '魚肉食譜',
  '備餐',
  '金目鱸魚',
  '金目鱸魚片',
  '金目鱸魚清肉排',
  '術後飲食',
  '手術飲食',
  '鱸魚湯',
  '台灣魚種'],
 51: ['飲食控制',
  '瘦身食譜',
  '健身食譜',
  '增肌減脂',
  '低卡飲食',
  '高蛋白飲食',
  '輕食選擇',
  '即食雞胸',
  '低醣飲食',
  '優質蛋白'],
 56: ['早餐美食',
  '點心推薦',
  '餅乾零食',
  '手工蔥餅',
  '美味蔥抓餅',
  '精選蔥餅',
  '新鮮蛋餅皮',
  '雲林蔥餅',
  '香氣濃郁的餅乾',
  '蔥味香濃@SpringBootApplication'],
 76: ['魚肉料理', '魚肉食譜', '備餐', '魚排', '鱈魚', '比目魚', '白肉魚', '炸魚', '大比目魚', '魚肉塊'],
 77: ['沙蟹', '海鮮', '花蟹', '螃蟹', '備餐', '蟹料理', '海鮮料理', '東海花蟹', '東海鮮甜沙蟹身', '快炒料理'],
 79: ['炸魚', '魚塊', '魚排', '宵夜', 

In [13]:
rows = []
evaluator = final.TagsEvaluator(product_tags)
for split, metrics in evaluator.cal_ranking_metrics().items():
    metrics['name'] = split
    rows.append(metrics)

pd.DataFrame(rows).set_index('name')

Dropping 63 terms


,hr@1,hr@5,hr@10,hr@20,mrr@1,mrr@5,mrr@10,mrr@20,ndcg@1,ndcg@5,ndcg@10,ndcg@20
name,,,,,,,,,,,,
train,0.757019,0.848590,0.887373,0.901251,0.757019,1.011441,1.036606,1.041177,0.757019,0.799963,0.818021,0.823577
val,0.781565,0.851141,0.891776,0.907557,0.781565,1.047759,1.077219,1.082898,0.781565,0.811211,0.830320,0.836916
test,0.773950,0.860602,0.898797,0.910790,0.773950,1.040070,1.066039,1.070201,0.773950,0.814014,0.832002,0.836996
